# Библиотеки

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
pd.options.display.float_format = '{:.3}'.format

import praw
import snscrape.modules.twitter as sntwitter
import datetime
import requests
from tqdm.notebook import tqdm
import cryptocompare
import getpass
import math
import numpy as np
import seaborn as sns
import scipy.stats as ss
import random
from collections import defaultdict

from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.metrics import mean_absolute_percentage_error, mean_squared_error
from statsmodels.stats.outliers_influence import variance_inflation_factor

import tensorflow as tf
from os import listdir
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Embedding
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras import layers
from keras.layers.advanced_activations import LeakyReLU
from tensorflow import keras
from ax.service.ax_client import AxClient
from ax.utils.notebook.plotting import render, init_notebook_plotting

from IPython.core.display import display, HTML
display(HTML('<style>.container {width: 100% !important; } </style>'))

# Reddit minutely

In [18]:
opt_x = pd.read_csv('instr minutely.csv',index_col=0).values.ravel()

X_total = pd.read_csv('reddit minutely total X.csv',usecols=opt_x)
Y = pd.read_csv('reddit minutely with sentiment.csv',index_col = 0)[['total_volume_total','returns']]

X_total = X_total.iloc[math.ceil(X_total.shape[0]*0.3):]
Y = Y.iloc[math.ceil(Y.shape[0]*0.3):][['returns']]

In [21]:
def abnormal_xgb(x,y,window):
    
    y = (y - y.rolling(window).mean()).dropna()
    x = x.iloc[window-1:]
    xg_reg = xgb.XGBRegressor()

    wild_window_error = []
    rolling_window_error = []
    
    wild_window_return = []
    rolling_window_return = []
    
    for i in tqdm(range(200,x.shape[0])):
        xg_reg.fit(x.iloc[i-200:i],y.iloc[i-200:i])
        preds = xg_reg.predict(pd.DataFrame(x.iloc[i]).T)
        rolling_window_error += [mean_absolute_percentage_error(pd.DataFrame(y.iloc[i]).T, preds)]
        rolling_window_return += [preds[0]]
        
        xg_reg.fit(x.iloc[:i],y.iloc[:i])
        preds = xg_reg.predict(pd.DataFrame(x.iloc[i]).T)
        wild_window_error += [mean_absolute_percentage_error(pd.DataFrame(y.iloc[i]).T, preds)]
        wild_window_return += [preds[0]]
        
    return {'rolling error':rolling_window_error,
           'rolling predinc':rolling_window_return,
           'window error':wild_window_error,
           'window predit':wild_window_return}

In [23]:
res_5 = abnormal_xgb(x=X_total,y=Y,window=5)
res_10 = abnormal_xgb(x=X_total,y=Y,window=10)
res_15 = abnormal_xgb(x=X_total,y=Y,window=15)
res_20 = abnormal_xgb(x=X_total,y=Y,window=20)
res_25 = abnormal_xgb(x=X_total,y=Y,window=25)
res_30 = abnormal_xgb(x=X_total,y=Y,window=30)

  0%|          | 0/1736 [00:00<?, ?it/s]

  0%|          | 0/1731 [00:00<?, ?it/s]

  0%|          | 0/1726 [00:00<?, ?it/s]

  0%|          | 0/1721 [00:00<?, ?it/s]

  0%|          | 0/1716 [00:00<?, ?it/s]

  0%|          | 0/1711 [00:00<?, ?it/s]

In [29]:
pd.DataFrame([res_5['window error'],res_10['window error'],res_15['window error'],res_20['window error'],res_25['window error'],res_30['window error']],
            index = ['window 5','window 10','window 15','window 20','window 25','window 30']).T.describe()

,window 5,window 10,window 15,window 20,window 25,window 30
count,1.74e+03,1.73e+03,1.73e+03,1.72e+03,1.72e+03,1.71e+03
mean,7.31,37.7,2.4,5.26,3.94,2.87
std,1.46e+02,1.26e+03,7.87,82.1,38.7,19.7
min,0.00558,0.00663,0.000756,0.00319,0.000448,0.00174
25%,0.806,0.798,0.84,0.819,0.818,0.803
50%,1.06,1.06,1.1,1.09,1.08,1.08
75%,1.57,1.54,1.63,1.62,1.59,1.55
max,5.48e+03,5.18e+04,2.03e+02,2.44e+03,1.11e+03,7.51e+02


In [30]:
pd.DataFrame([res_5['rolling error'],res_10['rolling error'],res_15['rolling error'],res_20['rolling error'],res_25['rolling error'],res_30['rolling error']],
            index = ['rolling 5','rolling 10','rolling 15','rolling 20','rolling 25','rolling 30']).T.describe()

,rolling 5,rolling 10,rolling 15,rolling 20,rolling 25,rolling 30
count,1.74e+03,1.73e+03,1.73e+03,1.72e+03,1.72e+03,1.71e+03
mean,2.48,12.9,2.73,3.89,4.32,3.0
std,8.8,2.52e+02,10.6,32.2,46.3,14.7
min,0.00474,0.000271,0.0013,0.000667,0.0011,0.000197
25%,0.781,0.773,0.8,0.769,0.789,0.775
50%,1.12,1.1,1.11,1.09,1.09,1.12
75%,1.74,1.7,1.73,1.74,1.74,1.8
max,2.06e+02,7.13e+03,3.41e+02,9.09e+02,1.57e+03,4.45e+02


# twitter

In [31]:
twitter = pd.read_csv('twitter total X.csv',index_col = 0)
intrs = pd.read_csv('instr twitter.csv',index_col=0).values.ravel()
X_all = twitter
X = twitter[intrs]
print('X:',X.shape)

twitter = pd.read_csv('twitter with PCA.csv',index_col = 0)
Y = twitter[['returns']]
print('Y:',Y.shape)

X: (70306, 209)
Y: (70306, 1)


## RNN

In [36]:
def abnormal_rnn(x,y,window):
    y = (y - y.rolling(window).mean()).dropna()
    x = x.iloc[window-1:]

    model = Sequential()
    model.add(Flatten())
    model.add(Dense(930, input_dim=930, activation='sigmoid'))
    model.add(Dense(700, activation='relu'))
    model.add(Dense(400, activation='relu'))
    model.add(Dense(200, activation='relu'))
    model.add(Dense(100, activation='relu'))
    model.add(Dense(50, activation='relu'))
    model.add(Dense(1, activation='relu'))
    model.compile(loss='mse',optimizer='adam', metrics=['mean_absolute_percentage_error'])
    
    model.fit(x.values, y.values.ravel(), epochs=10, batch_size=1000)
    _, accuracy = model.evaluate(x, y)
    print('Accuracy: %.2f' % (accuracy*100))

In [37]:
abnormal_rnn(x = X,y = Y, window = 5)

Epoch 1/10
71/71 [==============================] - 3s 36ms/step - loss: 0.0039 - mean_absolute_percentage_error: 18501.7852
Epoch 2/10
71/71 [==============================] - 2s 34ms/step - loss: 8.3260e-07 - mean_absolute_percentage_error: 99.9337
Epoch 3/10
71/71 [==============================] - 2s 34ms/step - loss: 8.3260e-07 - mean_absolute_percentage_error: 99.9337
Epoch 4/10
71/71 [==============================] - 2s 34ms/step - loss: 8.3260e-07 - mean_absolute_percentage_error: 99.9337
Epoch 5/10
71/71 [==============================] - 2s 34ms/step - loss: 8.3260e-07 - mean_absolute_percentage_error: 99.9337
Epoch 6/10
71/71 [==============================] - 3s 37ms/step - loss: 8.3260e-07 - mean_absolute_percentage_error: 99.9337
Epoch 7/10
71/71 [==============================] - 3s 41ms/step - loss: 8.3260e-07 - mean_absolute_percentage_error: 99.9337
Epoch 8/10
71/71 [==============================] - 3s 41ms/step - loss: 8.3260e-07 - mean_absolute_percentage_error: 9

In [38]:
abnormal_rnn(x = X,y = Y, window = 10)

Epoch 1/10
71/71 [==============================] - 3s 38ms/step - loss: 1.3208e-06 - mean_absolute_percentage_error: 102.2207
Epoch 2/10
71/71 [==============================] - 3s 41ms/step - loss: 9.3125e-07 - mean_absolute_percentage_error: 99.9464
Epoch 3/10
71/71 [==============================] - 3s 41ms/step - loss: 9.3125e-07 - mean_absolute_percentage_error: 99.9464
Epoch 4/10
71/71 [==============================] - 3s 41ms/step - loss: 9.3125e-07 - mean_absolute_percentage_error: 99.9464
Epoch 5/10
71/71 [==============================] - 3s 41ms/step - loss: 9.3125e-07 - mean_absolute_percentage_error: 99.9464
Epoch 6/10
71/71 [==============================] - 3s 40ms/step - loss: 9.3125e-07 - mean_absolute_percentage_error: 99.9464
Epoch 7/10
71/71 [==============================] - 3s 41ms/step - loss: 9.3125e-07 - mean_absolute_percentage_error: 99.9464
Epoch 8/10
71/71 [==============================] - 3s 41ms/step - loss: 9.3125e-07 - mean_absolute_percentage_error:

In [39]:
abnormal_rnn(x = X,y = Y, window = 15)

Epoch 1/10
71/71 [==============================] - 3s 39ms/step - loss: 8.0979e-05 - mean_absolute_percentage_error: 1115.3666
Epoch 2/10
71/71 [==============================] - 3s 42ms/step - loss: 9.6558e-07 - mean_absolute_percentage_error: 99.9565
Epoch 3/10
71/71 [==============================] - 3s 42ms/step - loss: 9.6558e-07 - mean_absolute_percentage_error: 99.9565
Epoch 4/10
71/71 [==============================] - 3s 41ms/step - loss: 9.6558e-07 - mean_absolute_percentage_error: 99.9565
Epoch 5/10
71/71 [==============================] - 3s 44ms/step - loss: 9.6558e-07 - mean_absolute_percentage_error: 99.9565
Epoch 6/10
71/71 [==============================] - 3s 41ms/step - loss: 9.6558e-07 - mean_absolute_percentage_error: 99.9565
Epoch 7/10
71/71 [==============================] - 3s 41ms/step - loss: 9.6558e-07 - mean_absolute_percentage_error: 99.9565
Epoch 8/10
71/71 [==============================] - 3s 41ms/step - loss: 9.6558e-07 - mean_absolute_percentage_error

In [40]:
abnormal_rnn(x = X,y = Y, window = 20)

Epoch 1/10
71/71 [==============================] - 3s 37ms/step - loss: 8.0207e-05 - mean_absolute_percentage_error: 5192.3301
Epoch 2/10
71/71 [==============================] - 3s 41ms/step - loss: 9.8307e-07 - mean_absolute_percentage_error: 99.9602
Epoch 3/10
71/71 [==============================] - 3s 41ms/step - loss: 9.8307e-07 - mean_absolute_percentage_error: 99.9602
Epoch 4/10
71/71 [==============================] - 3s 40ms/step - loss: 9.8307e-07 - mean_absolute_percentage_error: 99.9602
Epoch 5/10
71/71 [==============================] - 3s 40ms/step - loss: 9.8307e-07 - mean_absolute_percentage_error: 99.9602
Epoch 6/10
71/71 [==============================] - 3s 40ms/step - loss: 9.8307e-07 - mean_absolute_percentage_error: 99.9602
Epoch 7/10
71/71 [==============================] - 3s 41ms/step - loss: 9.8307e-07 - mean_absolute_percentage_error: 99.9602
Epoch 8/10
71/71 [==============================] - 3s 40ms/step - loss: 9.8307e-07 - mean_absolute_percentage_error

In [41]:
abnormal_rnn(x = X,y = Y, window = 25)

Epoch 1/10
71/71 [==============================] - 3s 35ms/step - loss: 0.0010 - mean_absolute_percentage_error: 10773.2871
Epoch 2/10
71/71 [==============================] - 3s 40ms/step - loss: 9.9230e-07 - mean_absolute_percentage_error: 99.9674
Epoch 3/10
71/71 [==============================] - 3s 41ms/step - loss: 9.9230e-07 - mean_absolute_percentage_error: 99.9674
Epoch 4/10
71/71 [==============================] - 3s 41ms/step - loss: 9.9230e-07 - mean_absolute_percentage_error: 99.9674
Epoch 5/10
71/71 [==============================] - 3s 40ms/step - loss: 9.9230e-07 - mean_absolute_percentage_error: 99.9674
Epoch 6/10
71/71 [==============================] - 3s 40ms/step - loss: 9.9230e-07 - mean_absolute_percentage_error: 99.9674
Epoch 7/10
71/71 [==============================] - 3s 40ms/step - loss: 9.9230e-07 - mean_absolute_percentage_error: 99.9674
Epoch 8/10
71/71 [==============================] - 3s 40ms/step - loss: 9.9230e-07 - mean_absolute_percentage_error: 9

In [42]:
abnormal_rnn(x = X,y = Y, window = 30)

Epoch 1/10
71/71 [==============================] - 3s 41ms/step - loss: 1.0338e-06 - mean_absolute_percentage_error: 100.7615
Epoch 2/10
71/71 [==============================] - 3s 43ms/step - loss: 9.9742e-07 - mean_absolute_percentage_error: 99.9760
Epoch 3/10
71/71 [==============================] - 3s 40ms/step - loss: 9.9742e-07 - mean_absolute_percentage_error: 99.9760
Epoch 4/10
71/71 [==============================] - 3s 40ms/step - loss: 9.9742e-07 - mean_absolute_percentage_error: 99.9760
Epoch 5/10
71/71 [==============================] - 3s 41ms/step - loss: 9.9742e-07 - mean_absolute_percentage_error: 99.9760
Epoch 6/10
71/71 [==============================] - 3s 40ms/step - loss: 9.9742e-07 - mean_absolute_percentage_error: 99.9760
Epoch 7/10
71/71 [==============================] - 3s 40ms/step - loss: 9.9742e-07 - mean_absolute_percentage_error: 99.9760
Epoch 8/10
71/71 [==============================] - 3s 40ms/step - loss: 9.9742e-07 - mean_absolute_percentage_error: